In [1]:
!pip install datasets
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [2]:
dataset_news = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [3]:
# всего 4 класса, по 25% каждого в dataset['train']
# сформирую датасет из 8000 примеров, по 2000 примеров на каждый класс

count0, count1, count2, count3 = 0, 0, 0, 0
dataset_short = []
for i in range(len(dataset_news['train'])):
  if dataset_news['train'][i]['label'] == 0 and count0 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_news['train'][i]['label'] == 1 and count1 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 1})
    count1 += 1
  elif dataset_news['train'][i]['label'] == 2 and count2 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 2})
    count2 += 1
  elif dataset_news['train'][i]['label'] == 3 and count3 < 3000:
    dataset_short.append({'news': dataset_news['train'][i]['text'], 'label': 3})
    count3 += 1
len(dataset_short)

12000

In [4]:
import random
random.shuffle(dataset_short)
dataset_news = {'train': dataset_short[:9600], 'test': dataset_short[9600:]}

In [5]:
# Выводим объём выборок
print('Объём train:', len(dataset_news['train']))
print('Объём test:', len(dataset_news['test']))

train_texts_news = [s['news'] for s in dataset_news['train']]
test_texts_news = [s['news'] for s in dataset_news['test']]

train_labels_news = [s['label'] for s in dataset_news['train']]
test_labels_news = [s['label'] for s in dataset_news['test']]

# ВЫводим примеры
print(train_texts_news[0], train_labels_news[0])
print(test_texts_news[0], test_labels_news[0])

Объём train: 9600
Объём test: 2400
Sri Lanka internet disruption said not caused by I Sri Lanka Telecom Thursday decided to withdraw its case against the Indian ship accused of severing the island #39;s main link to the internet after conclusive evidence that the vessel had nothing to do with the damage caused to submarine fibre optic cable  3
Two Bus Bombs in India Kill 4, Injure 39 GAUHATI, India Aug. 26, 2004 - A passenger bus and another carrying paramilitary soldiers and their families were blown up in separate explosions in the insurgency-wracked Assam state on Thursday, killing  0


In [6]:
# второй датасет
dataset_imdb = load_dataset("imdb")

count0, count1 = 0, 0
dataset_short_imdb = []

for i in range(len(dataset_imdb['train'])):
  if dataset_imdb['train'][i]['label'] == 0 and count0 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 0})
    count0 += 1
  elif dataset_imdb['train'][i]['label'] == 1 and count1 < 6000:
    dataset_short_imdb.append({'text': dataset_imdb['train'][i]['text'], 'label': 1})
    count1 += 1
len(dataset_short_imdb)

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

12000

In [7]:
random.shuffle(dataset_short_imdb)
dataset_imdb = {'train': dataset_short_imdb[:9600], 'test': dataset_short_imdb[9600:]}

In [8]:
# Выводим объём выборок
print('Объём train:', len(dataset_imdb['train']))
print('Объём test:', len(dataset_imdb['test']))

train_texts_imdb = [s['text'] for s in dataset_imdb['train']]
test_texts_imdb = [s['text'] for s in dataset_imdb['test']]

train_labels_imdb = [s['label'] for s in dataset_imdb['train']]
test_labels_imdb = [s['label'] for s in dataset_imdb['test']]

print(train_texts_imdb[0])
print(train_labels_imdb[0])
print()
print(test_texts_imdb[0])
print(test_labels_imdb[0])

Объём train: 9600
Объём test: 2400
Although there are some snippets in this 4-part documentary hinting at the necessity for recreational drug law reform, these are not very well-developed, in contrast to the many snippets from those who feel that the drugs that happen to currently be illegal are a scourge for which the only imaginable solution is incarceration of even those who are guilty even of mere possession of such drugs.<br /><br />Although this program, as a whole, leaves the viewer with the impression that the drug war is largely a futile exercise and a waste of money, and for that it deserves some praise, almost nothing in this documentary addresses the very real problems that total war against those who merely possess illegal drugs obviously causes and contributes to--very real problems that most drug warriors themselves would tell you, if asked, they think the drug war is designed to solve. For example, while many minutes are spent on the surge in violence associated with th

In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.metrics import f1_score
stop_words = list(set(stopwords.words("english")))

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [10]:
def base_preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Zа-яА-Я ]', '', text)
    tokens = word_tokenize(text)
    return tokens

In [11]:
def stopwords_special(text):
  tokens = base_preprocessing(text)
  tokens = [word for word in tokens if word not in stop_words]  # убираем стоп-слова
  return tokens

In [12]:
def stemming(text):
  tokens = stopwords_special(text)

  stemmer = nltk.PorterStemmer()  # инициализируем стеммер
  stemmed_tokens = [stemmer.stem(token) for token in tokens]  # перебираем токены и применяем алгоритм стемминга
  return stemmed_tokens

In [13]:
def lemmatisation(text):
  tokens = stopwords_special(text)

  lemma = nltk.WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(token) for token in tokens]
  return lemma_tokens

In [14]:
def vectorize(train_texts, test_texts, method, edit):
  if method == "count":
    vectorizer = CountVectorizer()
  else:
    vectorizer = TfidfVectorizer()

  # edit = 1 (stopwords_specoal), 2 (stemming), 3 (lemmatisation)
  if edit == 1:
    train_texts = [' '.join(stopwords_special(text)) for text in train_texts]
    test_texts = [' '.join(stopwords_special(text)) for text in test_texts]
  elif edit == 2:
    train_texts = [' '.join(stemming(text)) for text in train_texts]
    test_texts = [' '.join(stemming(text)) for text in test_texts]
  elif edit == 3:
    train_texts = [' '.join(lemmatisation(text)) for text in train_texts]
    test_texts = [' '.join(lemmatisation(text)) for text in test_texts]

  train_text = vectorizer.fit_transform(train_texts)
  test_text = vectorizer.transform(test_texts)

  return train_text, test_text

In [16]:
results = []
for k in ['linear', 'poly']:  # ядро - разделяющая гиперплокость
  for c in [0.1, 9]:  # коэффициент регуляризации - "мягкость" границ
    for d in ['news', 'imdb']:
      for edit in [1, 2, 3]:
        if d == 'news':
          train_texts = train_texts_news
          test_texts = test_texts_news
          train_labels = train_labels_news
          test_labels = test_labels_news
        else:
          train_texts = train_texts_imdb
          test_texts = test_texts_imdb
          train_labels = train_labels_imdb
          test_labels = test_labels_imdb


        train_text, test_text = vectorize(train_texts, test_texts, 'tfidf', edit)
        svm_clf = SVC(kernel=k, C=c)
        svm_clf.fit(train_text, train_labels)
        svm_pred = svm_clf.predict(test_text)

        if edit == 1:
          results.append([k, c, d, 'stopwords_special', f1_score(test_labels, svm_pred, average='macro')])
        elif edit == 2:
          results.append([k, c, d, 'stemming', f1_score(test_labels, svm_pred, average='macro')])
        elif edit == 3:
          results.append([k, c, d, 'lemmatisation', f1_score(test_labels, svm_pred, average='macro')])

        if len(results) % 10 == 0:
          print(len(results))

10
20


In [18]:
df_results1 = pd.DataFrame(results, columns=['Разделяющая гиперплоскость', "Коэф-т регуляризации - 'мягкость' границ", "Dataset", "Preprocess", "F1-score"])
df_results1.to_excel("results_svd.xlsx", index=False)

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

results_lsa = []
for k_lsa in [10, 50, 100]:
  for k in ['linear', 'poly']:  # ядро - разделяющая гиперплокость
    for c in [0.1, 9]:  # коэффициент регуляризации - "мягкость" границ
      for d in ['news', 'imdb']:
        for edit in [1, 2, 3]:
          if d == 'news':
            train_texts = train_texts_news
            test_texts = test_texts_news
            train_labels = train_labels_news
            test_labels = test_labels_news
          else:
            train_texts = train_texts_imdb
            test_texts = test_texts_imdb
            train_labels = train_labels_imdb
            test_labels = test_labels_imdb


          train_text, test_text = vectorize(train_texts, test_texts, 'tfidf', edit)

          svd = TruncatedSVD(n_components=k_lsa, random_state=42)
          lsa = make_pipeline(svd)
          train_lsa = lsa.fit_transform(train_text)
          test_lsa = lsa.transform(test_text)

          train_lsa = np.clip(train_lsa, 0, None)
          test_lsa = np.clip(test_lsa, 0, None)
          svm_clf = SVC(kernel=k, C=c)
          svm_clf.fit(train_text, train_labels)
          svm_pred = svm_clf.predict(test_text)

          if edit == 1:
            results_lsa.append([k_lsa, k, c, d, 'stopwords_special', f1_score(test_labels, svm_pred, average='macro')])
          elif edit == 2:
            results_lsa.append([k_lsa, k, c, d, 'stemming', f1_score(test_labels, svm_pred, average='macro')])
          elif edit == 3:
            results_lsa.append([k_lsa, k, c, d, 'lemmatisation', f1_score(test_labels, svm_pred, average='macro')])

          if len(results_lsa) % 10 == 0:
            print(len(results_lsa))

10
20
30
40
50
60
70


In [16]:
len(results_lsa)

72

In [17]:
df_results1 = pd.DataFrame(results_lsa, columns=['k','Разделяющая гиперплоскость', "Коэф-т регуляризации - 'мягкость' границ", "Dataset", "Preprocess", "F1-score"])
df_results1.to_excel("results_svd_lsa.xlsx", index=False)